In [ ]:
import yfinance as yf
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
import seaborn as sns
from tqdm import tqdm
import pandas as pd
from statsmodels.tools.sm_exceptions import ValueWarning, HessianInversionWarning, ConvergenceWarning
import warnings
import csv


#in practice do not supress these warnings, they carry important information about the status of your model
warnings.filterwarnings('ignore', category=ValueWarning)
warnings.filterwarnings('ignore', category=HessianInversionWarning)
warnings.filterwarnings('ignore', category=ConvergenceWarning)


# Simulate Buying and Selling Stock Using ARMA(p,q)

In [ ]:
def run_simulation(returns, prices, amt, order, thresh, verbose=False, plot=True):
    if type(order) == float:
        thresh = None
        
    curr_holding = False
    events_list = []
    init_amt = amt

    #go through dates
    for date, r in tqdm (returns.iloc[14:].items(), total=len(returns.iloc[14:])):
        #if you're currently holding the stock, sell it
        if curr_holding:
            sell_price = prices.loc[date]
            curr_holding=False
            ret = (sell_price-buy_price)/buy_price
            amt *= (1+ret)
            events_list.append(('s', date, ret))
            
            if verbose:
                print('Sold at $%s'%sell_price)
                print('Predicted Return: %s'%round(pred,4))
                print('Actual Return: %s'%(round(ret, 4)))
                print('=======================================')
            continue

        #get data til just before current date
        curr_data = returns[:date]
        
        if type(order) == tuple:
            try:
                #fit model
                model = ARIMA(curr_data, order=order).fit()

                #get forecast
                pred = model.forecast().values[0]

            except Exception as ex:
                print(str(ex))
                pred = thresh - 1



        #if you predict a high enough return and not holding, buy stock
        if (not curr_holding) and \
        ((type(order) == float and np.random.random() < order) 
         or (type(order) == tuple and pred > thresh)
         or (order == 'last' and curr_data[-1] > 0)):
            
            curr_holding = True
            buy_price = prices.loc[date]
            events_list.append(('b', date))
            if verbose:
                print('Bought at $%s'%buy_price)
                
    if verbose:
        print('Total Amount: $%s'%round(amt,2))
        
    #graph
    if plot:
    
        plt.figure(figsize=(10,4))
        plt.plot(prices[14:])

        y_lims = (int(prices.min()*.95), int(prices.max()*1.05))
        shaded_y_lims = int(prices.min()*.5), int(prices.max()*1.5)

        for idx, event in enumerate(events_list):
            plt.axvline(event[1], color='k', linestyle='--', alpha=0.4)
            if event[0] == 's':
                color = 'green' if event[2] > 0 else 'red'
                plt.fill_betweenx(range(*shaded_y_lims), 
                                  event[1], events_list[idx-1][1], color=color, alpha=0.1)

        tot_return = round(100*(amt / init_amt - 1), 2)
        tot_return = str(tot_return) + '%'
        plt.title("%s Price Data\nThresh=%s\nTotal Amt: $%s\nTotal Return: %s"%(tickerSymbol, thresh, round(amt,2), tot_return), fontsize=20)
        plt.ylim(*y_lims)
        plt.show()
    
    return amt

In [ ]:
def run_simulation_last(returns, prices, amt, order, thresh, verbose=False, plot=True):
    if type(order) == float:
        thresh = None
        
    curr_holding = False
    events_list = []
    init_amt = amt

    #go through dates
    for date, r in tqdm (returns.iloc[14:].items(), total=len(returns.iloc[14:])):
        #if you're currently holding the stock, sell it
        if curr_holding:
            sell_price = prices.loc[date]
            curr_holding=False
            ret = (sell_price-buy_price)/buy_price
            amt *= (1+ret)
            events_list.append(('s', date, sell_price, ret))
            
            if verbose:
                print('Sold at $%s'%sell_price)
                print('Predicted Return: %s'%round(pred,4))
                print('Actual Return: %s'%(round(ret, 4)))
                print('=======================================')
            continue

        #get data til just before current date
        curr_data = returns[:date]
        
        if type(order) == tuple:
            try:
                #fit model
                model = ARIMA(curr_data, order=order).fit()

                #get forecast
                pred = model.forecast().values[0]

            except Exception as ex:
                print(str(ex))
                pred = thresh - 1



        #if you predict a high enough return and not holding, buy stock
        if (not curr_holding) and \
        ((type(order) == float and np.random.random() < order) 
         or (type(order) == tuple and pred > thresh)
         or (order == 'last' and curr_data[-1] > 0)):
            
            curr_holding = True
            buy_price = prices.loc[date]
            events_list.append(('b', date,buy_price))
            if verbose:
                print('Bought at $%s'%buy_price)
                
    if verbose:
        print('Total Amount: $%s'%round(amt,2))
        
    with open('profiles1.csv', 'a', newline='') as file:
        writer = csv.writer(file)

        writer.writerow([events_list[-1][1], events_list[-1][0],events_list[-1][2]])

    
    return amt

In [2]:
import csv
from datetime import datetime
from statsmodels.tsa.arima.model import ARIMA

import yfinance as yf
import csv
from datetime import datetime


# Step 2: Define the class with date and action attributes
class DataRecord:
    def __init__(self,action, buy_price, predicted_return, date, sell_price, total_amount: int, total_return: int):
        self.date = date
        self.total_amount = total_amount
        self.sell_price = sell_price
        self.total_return = total_return
        self.action = action
        self.buy_price = buy_price
        self.predicted_value = predicted_return


    def __repr__(self):
        return f"DataRecord(date={self.date}, action='{self.action}, buy_price='{self.buy_price}')"
    

def read_second_to_last_line(filename):
    with open(filename, newline='') as csvfile:
        reader = list(csv.reader(csvfile))
        if len(reader) < 3:
            return None  # If the file has less than 3 lines (including the header), return None
        return reader[-2]  # Second to last line
# Step 3: Read and serialize the CSV

def append_to_last_line(filename, additional_values):
    # Step 1: Read the entire CSV file
    with open(filename, 'r') as csvfile:
        reader = list(csv.reader(csvfile))
    
    # Step 2: Append the additional values to the last line
    if reader:
        reader[-1].extend(additional_values)
    
    # Step 3: Write the modified data back to the CSV file
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(reader)

def create_a_new_line(filename, values):
    # Step 3: Write the modified data back to the CSV file
    with open(filename, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(values)

def read_csv_serialize(filename):
    second_to_last_line = read_second_to_last_line(filename)
    # Parse the date

    # Extract the action
    action = second_to_last_line[0]
    buy_price = float(second_to_last_line[1])
    predicted_return = float(second_to_last_line[2])
    date = datetime.strptime(second_to_last_line[3], '%Y-%m-%d %H:%M:%S%z')
    sell_price = float(second_to_last_line[4])
    total_amount = float(second_to_last_line[5])
    total_return = float(second_to_last_line[6])

    # Create a DataRecord instance and append to the list
    return DataRecord(action, buy_price, predicted_return, date, sell_price, total_amount, total_return)

filename = 'profiles1.csv'  
order = (5,0,0)  
thresh = 0.005 
second_last = read_csv_serialize(filename)


tickerSymbol = 'AAPL'
data = yf.Ticker(tickerSymbol)

prices = data.history(period="3mo").Close
returns = prices.pct_change().dropna()


if second_last.action == 'b':
    sell_price = prices.values[-1]
    curr_holding=False
    ret = (sell_price-second_last.buy_price)/second_last.buy_price
    amt = (1+ret) * second_last.total_amount
    # Example usage
    additional_values = [prices.index[-1], sell_price, amt, ret]  # Values to append to the last line
    append_to_last_line(filename, additional_values)
else:
    additional_values = [prices.index[-1], -1, second_last.total_amount, second_last.total_return]  # Values to append to the last line
    append_to_last_line(filename, additional_values)
  

## prediction

if type(order) == tuple:
    try:
        #fit model
        a = returns[:second_last.date]
        model = ARIMA(a, order=order).fit()
        
        #get forecast
        pred = model.forecast().values[0]

    except Exception as ex:
        print(str(ex))
        pred = thresh - 1

if pred > thresh:
    create_a_new_line(filename, ('b', prices.values[-1], pred))
else:
    create_a_new_line(filename, ('n', prices.values[-1], pred))


/Users/zsomborveres-lakos/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/zsomborveres-lakos/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/zsomborveres-lakos/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/zsomborveres-lakos/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will 

# Read Data

In [ ]:
tickerSymbol = 'AAPL'
data = yf.Ticker(tickerSymbol)

In [ ]:
prices = data.history(period="3mo").Close
returns = prices.pct_change().dropna()

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(prices)
plt.ylabel('Prices', fontsize=20)

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(returns)
plt.ylabel('Return', fontsize=20)

In [ ]:
plot_acf(returns)
plt.show()

In [ ]:
plot_pacf(returns)
plt.show()

# Baseline Model : Random Buying 

In [ ]:
run_simulation(returns, prices, 100, 0.5, None, verbose=False)

In [ ]:
final_amts = [run_simulation(returns, prices, 100, 0.5, None, verbose=False, plot=False) for _ in range(1000)]

In [ ]:
plt.figure(figsize=(10,4))
sns.distplot(final_amts)
plt.axvline(np.mean(final_amts), color='k', linestyle='--')
plt.axvline(100, color='g', linestyle='--')
plt.title('Avg: $%s\nSD: $%s'%(round(np.mean(final_amts),2), round(np.std(final_amts),2)), fontsize=20)

# If Last Return was Positive, Buy

In [ ]:
run_simulation_last(returns, prices, 100, 'last', None, verbose=False)

# Try AR(1) Model

In [ ]:
for thresh in [0, 0.001, 0.005]:
    run_simulation(returns, prices, 100, (1,0,0), thresh, verbose=False)

# Try AR(5) Model

In [ ]:
for thresh in [0, 0.001, 0.005]:
    run_simulation(returns, prices, 100, (5,0,0), thresh, verbose=False)

# Try ARMA(5,5) Model

In [ ]:
for thresh in [0, 0.001, 0.005]:
    run_simulation(returns, prices, 100, (5,0,5), thresh, verbose=False)